# Direct location SAR

## PyRugged API - tutorial 5


This tutorial explains how to initialize `PyRugged` and use it to geolocate a satellite SAR image. The antenna will be modeled by the SAR sensor and will be pointing right for the example. GPS and AOCS auxiliary data are available and provide us with a list of positions, velocities and attitude quaternions recorded during the acquisition. By passing all this information to `PyRugged`, we will be able to precisely locate each point of the image on the Earth. We will locate the image on an constant elevation ellipsoid above Earth surface. The objective here is limited to explain how to initialize everything `PyRugged` needs to know about the SAR sensor and the acquisition. This notebook is based on RADASAT example.

---

### Module imports

In [1]:
import orekit
import os

import numpy as np
import pytest

from org.hipparchus.geometry.euclidean.threed import Vector3D
from org.orekit.frames import Frame
from org.orekit.frames import FramesFactory
from org.orekit.frames import Transform
from org.orekit.time import AbsoluteDate, TimeScalesFactory
from org.orekit.utils import (
    CartesianDerivativesFilter,
    PVCoordinates,
    TimeStampedPVCoordinates,
)

from pyrugged.bodies.body_rotating_frame_id import BodyRotatingFrameId
from pyrugged.bodies.ellipsoid_id import EllipsoidId
from pyrugged.configuration.init_orekit import init_orekit
from pyrugged.intersection.algorithm_id import AlgorithmId
from pyrugged.intersection.constant_elevation_algorithm import ConstantElevationAlgorithm
from pyrugged.location.sar import SARLocation
from pyrugged.model.pyrugged_builder import PyRuggedBuilder
from pyrugged.sar_sensor.doppler_model import DopplerModel
from pyrugged.sar_sensor.range_from_pixel import RangeFromPixel, RangeGridCreation
from pyrugged.sar_sensor.sar_line_datation import SARLineDatation
from pyrugged.sar_sensor.sar_sensor import SARSensor
from pyrugged.utils.constants import Constants
from pyrugged.utils.coordinates_reader import extract_pv_from_txt
from pyrugged.utils.spacecraft_to_observed_body_sar import SpacecraftToObservedBodySAR

### Initialization 

initialize JVM or GraalVM subtrat VM and Orekit, internal orekit data is used here. See configuration section of user manual for further information.

In [2]:
init_orekit()

## 1 - Sensor's definition

Let’s start by defining the SAR sensor. The sensor model is described by its datation model, range model and doppler model (which will be zero doppler as it's the only one implemented for now)

### Datation Model

We need to define the datation model for each pixel (col, line) of the image. In order to construct this model, will we construct a grid containing dates which will be interpolated to compute the date for each pixel. For RADARSAT the date will be constant along lines that is to say that 2 pixels in the same line will have the same date. Here for the example line 0.5 was acquired at 2016-06-05T06:02:08.226323 and line 27193.5 at 2016-06-05T06:02:16.108620. We used pixel halves here as ESA RADARSAT products use this convention.

In [3]:
reference_date = AbsoluteDate("2016-06-05T06:02:08.226323", TimeScalesFactory.getUTC())

lines = [0.5, 27193.5]
pixels = [0, 20576]
corresponding_date_gap = [
        [0, -reference_date.durationFrom(AbsoluteDate("2016-06-05T06:02:16.108620", TimeScalesFactory.getUTC()))],
        [0, -reference_date.durationFrom(AbsoluteDate("2016-06-05T06:02:16.108620", TimeScalesFactory.getUTC()))],
]

The SARLineDatation model is instanciated with 4 arguments: the first is the pixel numbers used to initialise the grid, the second is the line numbers used to initialise the grid, the third a reference date with which a difference will be done to compute a grid of durations compared to this reference date and the last argument is the grid of time differences between (pixel,line) date and reference date.

1st arg : pixel column

2nd arg : pixel line

3rd arg : reference date

4th arg : date offset with reference date grid for each (pixel column, pixel line)

In [4]:
sar_datation = SARLineDatation(np.array(pixels), np.array(lines), reference_date, np.array(corresponding_date_gap))

### Range Model

We need to define the range model for each pixel (col, line) of the image. In order to construct this model, will we construct a grid containing range which will be interpolated to compute the range for each pixel. For RADARSAT the range is given by a polynom applied to the pixel column. The polynom will be very simple if the SAR product is given in SLC (basically order 1) and will be much more complex if the SAR product is given in GRD. For each col, the polynom expression is :

$distSR_{col} = \sum_{i=0}^{n} coeff_i * (col ∗ sampledPixelSpacing – groundRangeOrigin)^i$

Where polynome coefficients are given in RADARSAT product.

**Warning n°1** : the col value in the previous formula varies from 1 to total pixel number, but to be compatible with the code and python starting at 0, you must have pixel number varying from 0 to total pixel number - 1 when you construct the range model.

**Warning n°2** : a parameter available in the RADARSAT product will tell you if the pixel ordering time is increasing or decreasing. If increasing distSR_col correspond to column col, if decreasing distSR_col correspond to total_col - col + 1. This is handle by the range_model implementation when calling ground_range_to_slant_range_polynom_application.

**Warning n°3** : RADARSAT product use a convention for which location are given for pixel halves, this is taken into account into the range_model.

For an image with 20577 pixels and 27194 lines and for which pixel ordering time is decreasing, we use the class RangeGridCreation which take polynome coefficient and total number of pixels to be able to compute distSR polynome for given column :

In [5]:
# Pixel time ordering : decreasing
pixel_time_increasing = False

# Polynome coefficient for RADARSAT SLC product
sr_coefficient = np.array([9.738835958311997e05, 1.0, 0.0, 0.0, 0.0, 0.0])[::-1]

# Pixel number and ground range origin
total_pixel_number = 20577
ground_range_or = 0.0

# Pixel size = sampled pixel spacing
pixel_size_slc = 1.33117902

# Range Grid Creation class to construct and compute distSR for given column. Only useable for RADARSAT data
# not useful for Sentinal data
range_grid_construction = RangeGridCreation(total_pixel_number, sr_coefficient)
ranges = [
    [
        range_grid_construction.ground_range_to_slant_range_polynom_application(
            pixel_size_slc, ground_range_or, 0, pixel_time_increasing
        ),
        range_grid_construction.ground_range_to_slant_range_polynom_application(
            pixel_size_slc, ground_range_or, 0, pixel_time_increasing
        ),
    ],
    [
        range_grid_construction.ground_range_to_slant_range_polynom_application(
            pixel_size_slc, ground_range_or, 20576, pixel_time_increasing
        ),
        range_grid_construction.ground_range_to_slant_range_polynom_application(
            pixel_size_slc, ground_range_or, 20576, pixel_time_increasing
        ),
    ],
]

The RangeFromPixel model is instanciated with 3 arguments: the first is the pixel numbers used to initialise the grid, the second is the line numbers used to initialise the grid and the last argument is the grid of range for (pixel,line).

1st arg : pixel column

2nd arg : pixel line

4th arg : range corresponding to each (pixel column, pixel line) computed with the distSR polynome

In [6]:
range_pix_correspondance = RangeFromPixel(np.array(pixels), np.array(lines), np.array(ranges))

### Doppler Model

For now only the zero doppler model as been implemented, that is to say that doppler contribution will always be zero and is easy to initialize

In [7]:
doppler = DopplerModel()

### SAR Sensor

Now that datation model, range model and doppler model have been defined, we can construct our SAR Sensor

In [8]:
# In our example the SAR antenna is pointing right
antenna_pointing_right = True

sensor = SARSensor('sar_sensor', sar_datation, range_pix_correspondance, doppler, antenna_pointing_right)

The first parameter is the nickname of the sensor. It is necessary because we can define multiple sensors. The last argument is to say if the antenna is poiting right or left.

## 2 - Satellite position, velocity and attitude

As input to Rugged, we will need to pass sufficient information to describe the position of the platform during the acquisition, for SAR images the attitude of the satellite as no importance that is why we will consider all quaternion nul. In our example, the list of positions and velocities are hard-coded. In real life, we would extract GPS data from the satellite auxiliary telemetry.

Note that for simulation purpose, we could also use Orekit to simulate the orbit. It offers very convenient functions to propagate sun-synchronous orbits with yaw steering compensation (typical orbits for Earth Observation satellites).

### Reference frames

In our application, we simply need to know the name of the frames we are working with. Positions (unit: m) and velocities (unit: m/s) are given in the ITRF terrestrial frame.

### Satellite attitude

As said before, for SAR image satellite attitude is not important that is why no quaternion needed.

### Positions and velocities

Positions and velocities will be set in a list of TimeStampedPVCoordinates expressed in ITRF.

In [9]:
def add_satellite_pv(satellite_pv_list, abs_date, p_x, p_y, p_z, v_x, v_y, v_z):
    """Add element to satellite_pv_list"""

    date = AbsoluteDate(abs_date, TimeScalesFactory.getUTC())
    position = Vector3D(p_x, p_y, p_z)
    velocity = Vector3D(v_x, v_y, v_z)    
    satellite_pv_list.append(TimeStampedPVCoordinates(date, position, velocity))


satellite_pv_list = []

add_satellite_pv(satellite_pv_list, "2016-06-05T06:02:08.226323", 4.730625771849658e+06, 7.802107170901060e+05, 5.324424000376088e+06, 5.664882718527716e+03, -1.104946622266712e+03, -4.859206922710340e+03)
add_satellite_pv(satellite_pv_list, "2016-06-05T06:02:10.196897", 4.741778639440039e+06, 7.780301064799653e+05, 5.314837373988666e+06, 5.654523249054796e+03, -1.108225130750053e+03, -4.870570311536359e+03)
add_satellite_pv(satellite_pv_list, "2016-06-05T06:02:12.167472", 4.752911074320325e+06, 7.758430417126168e+05, 5.305228370472910e+06, 5.644139150101179e+03, -1.111496038016211e+03, -4.881913263246337e+03)
add_satellite_pv(satellite_pv_list, "2016-06-05T06:02:14.138046", 4.764023016689875e+06, 7.736495399943315e+05, 5.295597039874533e+06, 5.633730482873204e+03, -1.114759321316162e+03, -4.893235722740817e+03)
add_satellite_pv(satellite_pv_list, "2016-06-05T06:02:16.108620", 4.775114423833228e+06, 7.714496152625930e+05, 5.285943417785262e+06, 5.623297282833151e+03, -1.118014960336563e+03, -4.904537643655130e+03)

## 3 - PyRugged initialization

`PyRugged` object contains platform (frame transform between spacecraft,inertial and body frame), instrument and body elements.
As the Rugged top level object that will be used for all user interaction is quite involved and can be built in several different ways, a builder pattern approach has been adopted. The builder itself is configured by calling dedicated setters for each concept (Intersection algorithm, trajectory, …). As all these concepts can be chained together, the setters themselves implement the fluent interface pattern, which implies each setter returns the builder instance, and therefore another setter can be called directly.

__Setters__


- The `set_ellipsoid` setter defines the shape and orientation of the ellipsoid. We use simple predefined enumerates: `EllipsoidId.WGS84`, `InertialFrameId.EME2000`, but could also use a custom ellipsoid if needed.

- The `set_time_span` setter is used to define the global time span of the search algorithms (direct and inverse location). Four parameters are used for this: `acquisition_start_date`, `acquisition_stop_date`, `t_step` (step at which the pre-computed frames transforms cache will be filled), `time_tolerance` (margin allowed for extrapolation during inverse location, in seconds. The `t_step` parameter is a key parameter to achieve good performances as frames transforms will be precomputed throughout the time span using this time step. These computation are costly because they involve Earth precession/nutation models to be evaluated. So the transformed are precomputed and cached instead of being recomputed for each pixel. However, if direct and inverse location are expected to be performed over a reduced time span (say a few tens of seconds), precomputing the transforms over half an orbit at one millisecond rate would be a waste of computing power. Typical values are therefore to restrict the time span as much as possible to properly cover the expected direct and inverse location calls, and to use a step between one millisecond and one second, depending on the required accuracy. The exact value to use is mission-dependent. The final `time_tolerance` parameter is simply a margin used before and after the final precomputed transforms to allow a slight extrapolation if during a search the interval is slightly overshoot. A typical value is to allow a few images lines so for example a 5 lines tolerance would imply computing the tolerance as: `time_tolerance = 5 / line_sensor.get_rate(0))`.

- The `set_trajectory` setter defines the spacecraft evolution. The arguments are the list of time-stamped positions and velocities as well as the inertial frame with respect to which they are defined and options for interpolation: number of points to use and type of filter for derivatives. The interpolation polynomials for `nb_pv_points` without any derivatives (case of `CartesianDerivativesFilter.USE_P`: only positions are used, without velocities) have a degree `nb_pv_points - 1`. In case of computation with velocities included (case of `CartesianDerivativesFilter.USE_PV`), the interpolation polynomials have a degree `2 * nb_pv_points - 1`. If the positions/velocities data are of good quality and separated by a few seconds, one may choose only a few points but interpolate with both positions and velocities; in other cases, one may choose more points but interpolate only with positions. We find similar arguments for the attitude quaternions.

__Sensors__

The last setter used, `add_sensor`, registers a sensor (rather line_sensor or sar_sensor). As can be deduced from its prefix (add instead of set), it can be called several time to register several sensors that will all be available in the built Rugged instance. We have called the method only once here, so we will use only one sar sensor.

__Building PyRugged instance__

After the last setter has been called, we call the `build()` method which really build the `PyRugged` instance (and not a `PyRuggedBuilder` instance has the setter did).

The various setters can be called in any order. The only important thing is that once a `PyRugged` instance has been created by calling the `build()` method, it is independent from the builder so later calls to setters will not change the build instance. In fact, it is possible to create a builder, then call its `build()` method to create a first `PyRugged` instance, and then to modify the builder configuration by calling again some of the setters and building a second `PyRugged` instance from the new configuration. This allows to perform comparisons between two different configurations in the same program and without having to recreate everything.

In [10]:
builder = PyRuggedBuilder()


builder.set_ellipsoid(
    new_ellipsoid = None,
    ellipsoid_id = EllipsoidId.WGS84,
    body_rotating_frame_id = BodyRotatingFrameId.ITRF,
)

builder.set_time_span(
    AbsoluteDate("2016-06-05T06:02:08.226323", TimeScalesFactory.getUTC()),
    AbsoluteDate("2016-06-05T06:02:16.108620", TimeScalesFactory.getUTC()),
    0.01, 5.0
)

builder.set_trajectory(
    satellite_pv_list,
    5, CartesianDerivativesFilter.USE_PV
)

builder.add_sensor(sensor)

rugged = builder.build()

## 5 - Direct location of a single pixel

We now construct the SARLocation class to be able to locate our pixel on a altitude of 200m let's say


In [11]:
algorithm = ConstantElevationAlgorithm(200)
sar_location = SARLocation(rugged, algorithm)

Finally everything is set to do some real work. Let’s try to locate a point on Earth for upper left point (first line, first pixel):

In [12]:
coord_image = [0, 0]
direct_loc = sar_location.direct_location([0], [0], 'sar_sensor')

print("longitude {:.8f}° latitude {:.8f}° altitude {:.2f}m".format(np.degrees(direct_loc[0][0]), np.degrees(direct_loc[1][0]),direct_loc[2][0]))


longitude 1.80822026° latitude 49.30171247° altitude 200.00m
